# Build `MESH` Model Setup for the Bow River at Calgary

We use the `MESHFlow` Python package to build a `MESH` model setup for the Bow River at Calgary.

It's also better to keep the geospatial fabric files next to the "agnostic" step's outputs:

Let's start by importing the necessary libraries:

In [1]:
# import necessary libraries
import meshflow # version v0.1.0-dev1

import os # python 3.10.2

Now, let's provide necessary information to set up the `MESH` model:

In [2]:
# define home directory
home=os.getenv('HOME')

# main work path - modify
work_path = f'{home}/scratch/alliancecan-training-data/modelling/1-outputs/'

# using meshflow==v0.1.0-dev1
# modify each segment to match your settings
config = {
    'riv': os.path.join(work_path, 'geofabric-outputs', 'bcalgary_rivers.shp'),
    'cat': os.path.join(work_path, 'geofabric-outputs', 'bcalgary_subbasins.shp'),
    'landcover': os.path.join(work_path, 'gistool-outputs', 'bcalgary_stats_NA_NALCMS_landcover_2020_30m.csv'),
    'forcing_files': os.path.join(work_path, 'easymore-outputs'),
    'forcing_vars': [ # does the variable list, match those of the "agnostic" step?
        "RDRS_v2.1_P_P0_SFC",
        "RDRS_v2.1_P_HU_09944",
        "RDRS_v2.1_P_TT_09944",
        "RDRS_v2.1_P_UVC_09944",
        "RDRS_v2.1_A_PR0_SFC",
        "RDRS_v2.1_P_FB_SFC",
        "RDRS_v2.1_P_FI_SFC",
    ],
    'forcing_units': { # Here, enter RDRS's original variable units
        'RDRS_v2.1_P_P0_SFC': 'millibar',
        'RDRS_v2.1_P_HU_09944': 'kg/kg',
        'RDRS_v2.1_P_TT_09944': 'celsius',
        'RDRS_v2.1_P_UVC_09944': 'knot',
        'RDRS_v2.1_A_PR0_SFC': 'm/hr',
        'RDRS_v2.1_P_FB_SFC': 'W/m^2',
        'RDRS_v2.1_P_FI_SFC': 'W/m^2',
    },
    'forcing_to_units': { # And here, the units that MESH needs to read
         'RDRS_v2.1_P_UVC_09944': 'm/s',
         'RDRS_v2.1_P_FI_SFC': 'W/m^2',
         'RDRS_v2.1_P_FB_SFC': 'W/m^2',
         'RDRS_v2.1_A_PR0_SFC': 'mm/s',
         'RDRS_v2.1_P_P0_SFC': 'pascal',
         'RDRS_v2.1_P_TT_09944': 'kelvin',
         'RDRS_v2.1_P_HU_09944': 'kg/kg',
    },
    'main_id': 'COMID', # what is the main ID of each river segment? Column name in the `cat` Shapefile
    'ds_main_id': 'NextDownID', # what is the downstream segment ID for each river segment? ditto.
    'landcover_classes': { # these are the classes defined for NALCMS-Landsat 2015 dataset. Is this accurate?
        0: 'Unknown',
        1: 'Temperate or sub-polar needleleaf forest',
        2: 'Sub-polar taiga needleleaf forest',
        3: 'Tropical or sub-tropical broadleaf evergreen forest',
        4: 'Tropical or sub-tropical broadleaf deciduous forest',
        5: 'Temperate or sub-polar broadleaf deciduous forest',
        6: 'Mixed forest',
        7: 'Tropical or sub-tropical shrubland',
        8: 'Temperate or sub-polar shrubland',
        9: 'Tropical or sub-tropical grassland',
        10: 'Temperate or sub-polar grassland',
        11: 'Sub-polar or polar shrubland-lichen-moss',
        12: 'Sub-polar or polar grassland-lichen-moss',
        13: 'Sub-polar or polar barren-lichen-moss',
        14: 'Wetland',
        15: 'Cropland',
        16: 'Barren lands',
        17: 'Urban',
        18: 'Water',
        19: 'Snow and Ice',
    },
    'ddb_vars': { # the stuff that MESH needs: slope, river length, etc... Let me know if there is any issues here!
        'slope': 'ChnlSlope',
        'lengthkm': 'ChnlLength',
        'Rank': 'Rank',
        'Next': 'Next',
        'landcover': 'GRU',
        'unitarea': 'GridArea',
        'landcover_names': 'LandUse',
    },
    'ddb_units': {
        'ChnlSlope': 'm/m',
        'ChnlLength': 'km', # is it in km or m? Please check the units of the Shapefile you created!
        'Rank': 'dimensionless',
        'Next': 'dimensionless',
        'GRU': 'dimensionless',
        'GridArea': 'km^2', # what was the unit of the GridArea, or Shape_Area in the `catchments` Shapefile?
        'LandUse': 'dimensionless',
    },
    'ddb_to_units': {
        'ChnlSlope': 'm/m',
        'ChnlLength': 'm', # This is what MESH needs, no need to change.
        'Rank': 'dimensionless',
        'Next': 'dimensionless',
        'GRU': 'dimensionless',
        'GridArea': 'm^2', # This is what MESH needs, no need to change.
        'LandUse': 'dimensionless',
    },
    'ddb_min_values': {
        'ChnlSlope': 1e-10, # in case there are 0s in the `rivers` Shapefile, we need minimums for certain variables
        'ChnlLength': 1e-3,
        'GridArea': 1e-3,
    },
    'gru_dim': 'NGRU', # change to `NGRU` for 'MESH>=r1860', keep for 'MESH<=1860', for example for r1813.
    'hru_dim': 'subbasin',
    'outlet_value': -9999,
}

We can build an "instance" of the workflow class:

In [3]:
exp1 = meshflow.MESHWorkflow(**config)

And, we can run it using:

In [4]:
exp1.run()

/home/kasra545/virtual-envs/hydrology-env/lib/python3.10/site-packages/xarray/core/accessor_dt.py:72: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  values_as_series = pd.Series(values.ravel(), copy=False)


Once the run is finished, we can checkout the forcing and drainage database file:

In [5]:
exp1.forcing

<xarray.Dataset>
Dimensions:                (subbasin: 169, time: 744)
Coordinates:
  * time                   (time) datetime64[ns] 1985-01-01T13:00:00 ... 1985...
  * subbasin               (subbasin) int32 71032409 71032292 ... 71027942
Data variables:
    RDRS_v2.1_P_P0_SFC     (subbasin, time) float64 7.672e+04 ... 8.769e+04
    RDRS_v2.1_P_HU_09944   (subbasin, time) float64 0.0005257 ... 0.0003989
    RDRS_v2.1_P_TT_09944   (subbasin, time) float64 249.9 249.8 ... 251.8 251.8
    RDRS_v2.1_P_UVC_09944  (subbasin, time) float64 0.8553 0.8735 ... 1.261
    RDRS_v2.1_A_PR0_SFC    (subbasin, time) float64 6.282e-07 4.422e-07 ... 0.0
    RDRS_v2.1_P_FB_SFC     (subbasin, time) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0
    RDRS_v2.1_P_FI_SFC     (subbasin, time) float64 180.3 187.0 ... 130.7 130.2
    lat                    (subbasin) float64 51.66 51.63 51.67 ... 51.16 51.1
    lon                    (subbasin) float64 -116.2 -116.1 ... -114.4 -114.2
    crs                    int64 1
Attributes:
    author:       University of Calgary
    license:      GNU General Public License v3 (or any later version)
    purpose:      Create forcing .nc file for MESH
    Conventions:  CF-1.6
    history:      Created using MESHFlow version v0.1.0.dev1

In [6]:
exp1.ddb

<xarray.Dataset>
Dimensions:     (subbasin: 169, NGRU: 13)
Coordinates:
  * subbasin    (subbasin) int64 71032409 71032292 ... 71027957 71027942
Dimensions without coordinates: NGRU
Data variables:
    ChnlSlope   (subbasin) float64 0.01669 0.01355 ... 0.001508 0.001798
    ChnlLength  (subbasin) float64 6.012e+03 493.8 ... 2.059e+04 2.815e+04
    Rank        (subbasin) int32 1 2 3 4 5 6 7 8 ... 163 164 165 166 167 168 169
    Next        (subbasin) int32 6 6 8 8 10 10 12 ... 166 166 168 168 169 169 0
    GRU         (subbasin, NGRU) float64 0.222 0.0 0.0 ... 0.02607 0.0 0.0
    GridArea    (subbasin) float64 6.579e+07 2.559e+07 ... 9.346e+07 1.531e+08
    LandUse     (NGRU) <U49 'Temperate or sub-polar needleleaf forest' ... 'D...
    lat         (subbasin) float64 51.66 51.63 51.67 51.67 ... 51.12 51.16 51.1
    lon         (subbasin) float64 -116.2 -116.1 -116.4 ... -114.4 -114.4 -114.2
    crs         int64 1
Attributes:
    author:       University of Calgary
    license:      GNU General Public License v3 (or any later version)
    purpose:      Create a drainage database .nc file for MESH
    featureType:  point

In [7]:
# create a directory for MESH setup
try:
    os.makedirs(os.path.join(work_path, 'MESH-bow-at-calgary'))
except FileExistsError:
    pass

# saving drainage database and forcing files
exp1.save(os.path.join(work_path, 'MESH-bow-at-calgary'))

/home/kasra545/virtual-envs/hydrology-env/lib/python3.10/site-packages/meshflow/core.py:401: UserWarning: MESH settings need manual adjustments. Automation is a work in progress...
  warnings.warn(


____

If there is any issue, open a ticket on [MESHFlow's GitHub Webpage](https://github.com/kasra-keshavarz/meshflow/issues). Once this Notebook runs, then have the setting files along with a `MESH` executable all in the `MESH-smm` directory. Please be mindful of the settings, and assure the settings are all accurate to run `MESH` properly.